# Keras Tuner

In [77]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import kerastuner as kt

In [78]:
print(tf.__version__)

2.1.0


In [79]:
df = pd.read_csv('../AirQualityIndex-Keras-Tuner/Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [80]:
X = df.iloc[:,:-1] # Independent features 
y = df.iloc[:,-1] # Dependent features

In [81]:
X.shape

(1093, 8)

In [82]:
X.head()

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [83]:
y.head()

0    219.720833
1    182.187500
2    154.037500
3    223.208333
4    200.645833
Name: PM 2.5, dtype: float64

In [84]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense

def model_builder(hp):
    
    model = keras.Sequential()
    
    hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
    model.add(keras.layers.Dense(units=hp_units,activation='relu'))
    model.add(keras.layers.Dense(10))
    
    hp_learning_rate = hp.Choice('learning_rate',values = [1e-2,1e-3,1e-4])
    model.compile(optmizer=keras.optimizers.Adam(learning_rate = hp_learning_rate),
                                                loss='mean_absolute_error',
                                                metrics=['val_mean_absolute_error'])
    return model
    

In [85]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

### Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available 
- RandomSearch
- Hyperband
- BayesianOptimization
- Sklearn


hypermodel,
    objective,
    max_epochs,
    factor=3,
    hyperband_iterations=1,
    seed=None,
    hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True

tuner = kt.Hyperband(model_builder,
                     objective='val_mean_absolute_error',
                    max_epochs=10,
                    factor=3,
                    directory='my_dir')

In [86]:
tuner = kt.RandomSearch(
    build_model,
    objective='mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

In [87]:
tuner.search_space_summary()

In [88]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [90]:
tuner.search(X_train.values, y_train.values,
             epochs=5,
             validation_data=(X_test.values, y_test.values))

Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 16s - loss: 107.1445 - mean_absolute_error: 107.14 - ETA: 1s - loss: 79.8635 - mean_absolute_error: 79.8635   - ETA: 0s - loss: 73.9264 - mean_absolute_error: 73.926 - ETA: 0s - loss: 70.9455 - mean_absolute_error: 70.945 - 1s 1ms/sample - loss: nan - mean_absolute_error: nan - val_loss: 65.6008 - val_mean_absolute_error: 65.6008
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 68.9117 - mean_absolute_error: 68.911 - ETA: 0s - loss: 68.3652 - mean_absolute_error: 68.365 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 334us/sample - loss: nan - mean_absolute_error: nan - val_loss: 66.5220 - val_mean_absolute_error: 66.5220
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: 73.3483 - mean_absolute_error: 73.348 - ETA: 0s - loss: 67.4621 - mean_absolute_error: 67.462 - ETA: 0s - loss: nan 

d:\anacondafiles\envs\deeplearning\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 33s - loss: 118.2929 - mean_absolute_error: 118.29 - ETA: 4s - loss: 107.0925 - mean_absolute_error: 107.0925 - ETA: 2s - loss: 109.5503 - mean_absolute_error: 109.550 - ETA: 0s - loss: nan - mean_absolute_error: nan          - 2s 3ms/sample - loss: nan - mean_absolute_error: nan - val_loss: 103.3609 - val_mean_absolute_error: 103.3609
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 121.7166 - mean_absolute_error: 121.716 - ETA: 0s - loss: 104.2616 - mean_absolute_error: 104.261 - ETA: 0s - loss: nan - mean_absolute_error: nan          - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 426us/sample - loss: nan - mean_absolute_error: nan - val_loss: 71.5232 - val_mean_absolute_error: 71.5232
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: 87.8848 - mean_absolute_error: 87.884 - ETA: 0s - loss: 72.9431 - mean_absolute_error: 72.943 - ETA: 0s 

Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 21s - loss: 116.1958 - mean_absolute_error: 116.19 - ETA: 1s - loss: nan - mean_absolute_error: nan           - ETA: 0s - loss: nan - mean_absolute_error: na - 1s 2ms/sample - loss: nan - mean_absolute_error: nan - val_loss: 87.3057 - val_mean_absolute_error: 87.3057
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 103.0412 - mean_absolute_error: 103.041 - ETA: 0s - loss: 84.4724 - mean_absolute_error: 84.4725  - ETA: 0s - loss: 79.1691 - mean_absolute_error: 79.169 - ETA: 0s - loss: 75.7278 - mean_absolute_error: 75.727 - 0s 401us/sample - loss: nan - mean_absolute_error: nan - val_loss: 68.0989 - val_mean_absolute_error: 68.0989
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: 76.4270 - mean_absolute_error: 76.427 - ETA: 0s - loss: 60.9952 - mean_absolute_error: 60.995 - ETA: 0s - loss: 64.2111 - mean_absolute_error: 64.211 - 0s 353us/sample 

Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 13s - loss: 70.7938 - mean_absolute_error: 70.79 - ETA: 1s - loss: nan - mean_absolute_error: nan         - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 1s 1ms/sample - loss: nan - mean_absolute_error: nan - val_loss: 63.2703 - val_mean_absolute_error: 63.2703
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 94.7162 - mean_absolute_error: 94.716 - ETA: 0s - loss: 67.7275 - mean_absolute_error: 67.727 - ETA: 0s - loss: 68.2655 - mean_absolute_error: 68.265 - 0s 253us/sample - loss: nan - mean_absolute_error: nan - val_loss: 63.3852 - val_mean_absolute_error: 63.3852
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 211us/sample - loss: nan - mean_absolute_error: nan - val_loss: 63.4251 - val_mean_absolute_error: 63

Train on 765 samples, validate on 328 samples
Epoch 1/5
765/765 [==============================] - ETA: 15s - loss: 134.9104 - mean_absolute_error: 134.91 - ETA: 1s - loss: nan - mean_absolute_error: nan           - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 1s 1ms/sample - loss: nan - mean_absolute_error: nan - val_loss: 63.1071 - val_mean_absolute_error: 63.1071
Epoch 2/5
765/765 [==============================] - ETA: 0s - loss: 67.3385 - mean_absolute_error: 67.338 - ETA: 0s - loss: 66.7490 - mean_absolute_error: 66.749 - ETA: 0s - loss: 70.1135 - mean_absolute_error: 70.113 - 0s 305us/sample - loss: nan - mean_absolute_error: nan - val_loss: 62.0210 - val_mean_absolute_error: 62.0210
Epoch 3/5
765/765 [==============================] - ETA: 0s - loss: 52.7181 - mean_absolute_error: 52.718 - ETA: 0s - loss: 63.0859 - mean_absolute_error: 63.085 - ETA: 0s - loss: 63.2776 - mean_absolute_error: 63.277 - ETA: 0s - loss: nan - mean_a

INFO:tensorflow:Oracle triggered exit


In [91]:
tuner.results_summary()

In [71]:
tuner.search(X_train.values, y_train.values,
             epochs=5,
            validation_data=X_test.values,y_test.values)

Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 21s - loss: 103.0716 - mean_absolute_error: 103.07 - ETA: 3s - loss: 99.9834 - mean_absolute_error: 99.9834   - ETA: 1s - loss: 86.9621 - mean_absolute_error: 86.962 - ETA: 0s - loss: 83.4764 - mean_absolute_error: 83.476 - ETA: 0s - loss: 79.4234 - mean_absolute_error: 79.423 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 1s 2ms/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 267us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 75.7126 - mean_absolute_error: 75.712 - ETA: 0s - loss: 62.3950 - mean_absolute_error: 62.395 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - l

d:\anacondafiles\envs\deeplearning\lib\site-packages\kerastuner\engine\metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 15s - loss: 112.3372 - mean_absolute_error: 112.33 - ETA: 0s - loss: nan - mean_absolute_error: nan           - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 1s 1ms/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: 71.6040 - mean_absolute_error: 71.604 - ETA: 0s - loss: 67.3798 - mean_absolute_error: 67.379 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 173us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 65.3010 - mean_absolute_error: 65.301 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 176us/sample - loss: nan - mean_absolute_error: nan
Epoch 4/7
765/765 [==============================] - ETA: 0s - loss: 43.7711 - mean_absolute_error: 43.771 - ETA: 0s

Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 20s - loss: 88.6679 - mean_absolute_error: 88.66 - ETA: 1s - loss: 432.6472 - mean_absolute_error: 432.647 - ETA: 0s - loss: nan - mean_absolute_error: nan          - 1s 1ms/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: 54.7780 - mean_absolute_error: 54.778 - ETA: 0s - loss: 72.1932 - mean_absolute_error: 72.193 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 193us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 74.6917 - mean_absolute_error: 74.691 - ETA: 0s - loss: 60.7344 - mean_absolute_error: 60.734 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 190us/sample - loss: nan - mean_absolute_error: nan
Epoch 4/7
765/765 [==============================] - ETA: 0s - loss: 52.1751 - mean_absolute_error: 52.175 - ETA: 0s - loss: 55.4984 - mean_absolute_

Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 12s - loss: 107.0630 - mean_absolute_error: 107.06 - ETA: 0s - loss: 81.3953 - mean_absolute_error: 81.3953   - ETA: 0s - loss: nan - mean_absolute_error: nan        - 1s 885us/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: 69.5325 - mean_absolute_error: 69.532 - ETA: 0s - loss: 66.4776 - mean_absolute_error: 66.477 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 199us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 75.8975 - mean_absolute_error: 75.897 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 241us/sample - loss: nan - mean_absolute_error: nan
Epoch 4/7
765/765 [==============================] - ETA: 0s - loss: 82.2518 - mean_absolute_error: 82.251

Train on 765 samples
Epoch 1/7
765/765 [==============================] - ETA: 20s - loss: 93.9442 - mean_absolute_error: 93.94 - ETA: 1s - loss: 183.8867 - mean_absolute_error: 183.886 - ETA: 0s - loss: nan - mean_absolute_error: nan          - ETA: 0s - loss: nan - mean_absolute_error: na - 1s 1ms/sample - loss: nan - mean_absolute_error: nan
Epoch 2/7
765/765 [==============================] - ETA: 0s - loss: 65.1980 - mean_absolute_error: 65.198 - ETA: 0s - loss: nan - mean_absolute_error: nan        - ETA: 0s - loss: nan - mean_absolute_error: na - 0s 209us/sample - loss: nan - mean_absolute_error: nan
Epoch 3/7
765/765 [==============================] - ETA: 0s - loss: 62.1478 - mean_absolute_error: 62.147 - ETA: 0s - loss: 69.8938 - mean_absolute_error: 69.893 - ETA: 0s - loss: 63.8551 - mean_absolute_error: 63.855 - ETA: 0s - loss: nan - mean_absolute_error: nan        - 0s 253us/sample - loss: nan - mean_absolute_error: nan
Epoch 4/7
765/765 [==============================] - 

INFO:tensorflow:Oracle triggered exit
